# Cours : Optimisation Combinatoire avec Python
## Passage de Julia à Python

Ce notebook vous guide dans l'utilisation de Python pour l'optimisation combinatoire, en particulier pour le problème du voyageur de commerce (TSP).

## 1. Introduction : Python vs Julia pour l'Optimisation

### Différences principales

| Aspect | Julia | Python |
|--------|-------|--------|
| **Performance** | Très rapide (JIT) | Plus lent, mais optimisé avec NumPy |
| **Bibliothèques** | JuMP, JuMPeR | PuLP, OR-Tools, scipy.optimize |
| **Syntaxe** | Proche de MATLAB | Plus verbeux mais très lisible |
| **Écosystème** | Spécialisé optimisation | Très large écosystème |

### Avantages de Python pour ce projet
- **NetworkX** : Manipulation de graphes excellente
- **tsplib95** : Lecture directe des fichiers TSP
- **PuLP** : Modélisation PLNE intuitive
- **Visualisation** : Matplotlib très puissant

## 2. Bibliothèques Essentielles

### Importations de base

In [ ]:
# Bibliothèques pour les données et calculs
import numpy as np
import pandas as pd

# Bibliothèques pour les graphes
import networkx as nx

# Bibliothèques pour l'optimisation
import pulp  # Programmation linéaire en nombres entiers (PLNE)
from scipy.optimize import linprog  # Optimisation linéaire continue

# Bibliothèques pour les instances TSP
import tsplib95

# Bibliothèques pour la visualisation
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch

# Utilitaires
import random
import time
from typing import List, Tuple, Dict, Optional

## 3. Manipulation des Instances TSP

### Chargement d'une instance

In [ ]:
# Importons les fonctions de notre module instance
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath('')))

from instance import chargerInstance, creerGraphe, afficherGraphe

# Charger une petite instance pour les tests
fichier_tsp = "../data/fri26.tsp"
probleme = chargerInstance(fichier_tsp)

# Afficher les informations de l'instance
print("Nom:", probleme.name)
print("Dimension:", probleme.dimension)
print("Type:", probleme.type)
print("Type de poids:", probleme.edge_weight_type)

### Création et manipulation du graphe NetworkX

In [ ]:
# Créer le graphe complet avec les distances
graphe = creerGraphe(probleme)

print(f"Nombre de nœuds: {graphe.number_of_nodes()}")
print(f"Nombre d'arêtes: {graphe.number_of_edges()}")
print(f"Type de graphe: {type(graphe)}")

# Accéder aux nœuds et arêtes
print("\nPremiers nœuds:", list(graphe.nodes())[:5])
print("\nPremières arêtes avec poids:", list(graphe.edges(data=True))[:5])

# Calculer la distance entre deux nœuds
if graphe.number_of_nodes() > 0:
    noeuds = list(graphe.nodes())
    if len(noeuds) >= 2:
        dist = graphe[noeuds[0]][noeuds[1]]['weight']
        print(f"\nDistance entre {noeuds[0]} et {noeuds[1]}: {dist}")

### Extraction de la matrice de distances

**En Julia**, vous aviez probablement une matrice directement. En Python, nous devons l'extraire du graphe.

In [ ]:
def obtenir_matrice_distances(graphe: nx.Graph) -> np.ndarray:
    """
    Convertit un graphe NetworkX en matrice de distances numpy.
    
    Args:
        graphe: Graphe NetworkX avec poids sur les arêtes
        
    Returns:
        Matrice numpy de taille (n, n) avec les distances
    """
    n = graphe.number_of_nodes()
    noeuds = sorted(graphe.nodes())
    matrice = np.zeros((n, n))
    
    for i, u in enumerate(noeuds):
        for j, v in enumerate(noeuds):
            if u == v:
                matrice[i, j] = 0
            else:
                matrice[i, j] = graphe[u][v]['weight']
    
    return matrice

# Exemple d'utilisation
matrice_dist = obtenir_matrice_distances(graphe)
print(f"Taille de la matrice: {matrice_dist.shape}")
print(f"\nPremière ligne (distances depuis le nœud 1):\n{matrice_dist[0, :10]}")
print(f"\nMatrice symétrique: {np.allclose(matrice_dist, matrice_dist.T)}")

## 4. Résolution Exacte : Programmation Linéaire en Nombres Entiers (PLNE)

### Modélisation avec PuLP (équivalent de JuMP en Julia)

**En Julia**, vous utilisiez probablement JuMP. En Python, **PuLP** est l'équivalent le plus proche.

In [ ]:
def resoudre_tsp_plne(matrice_dist: np.ndarray, limite_temps: int = 300) -> Tuple[List[int], float]:
    """
    Résout le TSP avec une formulation PLNE (modèle de Miller-Tucker-Zemlin).
    
    Args:
        matrice_dist: Matrice de distances (n x n)
        limite_temps: Temps maximum en secondes
        
    Returns:
        Tuple (chemin, coût_total)
    """
    n = len(matrice_dist)
    
    # Créer le problème
    prob = pulp.LpProblem("TSP", pulp.LpMinimize)
    
    # Variables de décision
    # x[i][j] = 1 si on va de i à j, 0 sinon
    x = pulp.LpVariable.dicts("arc", 
                              [(i, j) for i in range(n) for j in range(n) if i != j],
                              cat='Binary')
    
    # Variables u[i] pour les contraintes MTZ (éviter les sous-tours)
    u = pulp.LpVariable.dicts("u", range(n), lowBound=0, upBound=n-1, cat='Integer')
    
    # Fonction objectif : minimiser la distance totale
    prob += pulp.lpSum([matrice_dist[i][j] * x[(i, j)] 
                        for i in range(n) for j in range(n) if i != j])
    
    # Contraintes : chaque ville a exactement un successeur
    for i in range(n):
        prob += pulp.lpSum([x[(i, j)] for j in range(n) if i != j]) == 1
    
    # Contraintes : chaque ville a exactement un prédécesseur
    for j in range(n):
        prob += pulp.lpSum([x[(i, j)] for i in range(n) if i != j]) == 1
    
    # Contraintes MTZ pour éviter les sous-tours
    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                prob += u[i] - u[j] + n * x[(i, j)] <= n - 1
    
    # Résoudre
    prob.solve(pulp.PULP_CBC_CMD(timeLimit=limite_temps, msg=1))
    
    # Extraire la solution
    if pulp.LpStatus[prob.status] == 'Optimal':
        chemin = [0]  # Commencer à la ville 0
        actuel = 0
        
        while len(chemin) < n:
            for j in range(n):
                if j != actuel and pulp.value(x[(actuel, j)]) == 1:
                    chemin.append(j)
                    actuel = j
                    break
        
        # Calculer le coût total
        cout_total = sum(matrice_dist[chemin[i]][chemin[i+1]] 
                        for i in range(len(chemin)-1))
        cout_total += matrice_dist[chemin[-1]][chemin[0]]  # Retour au départ
        
        return chemin, cout_total
    else:
        return [], float('inf')

# Test sur une petite instance (peut être long pour des instances grandes)
print("⚠️  Résolution PLNE peut prendre du temps...")
print("Pour tester rapidement, utilisez une instance très petite (ex: burma14.tsp)")

## 5. Heuristiques Constructives

### Plus Proche Voisin (Nearest Neighbor)

**En Julia**, vous aviez probablement des fonctions similaires. En Python, la syntaxe est plus verbeuse mais très claire.

In [ ]:
def plus_proche_voisin(matrice_dist: np.ndarray, depart: int = 0) -> Tuple[List[int], float]:
    """
    Heuristique du plus proche voisin pour le TSP.
    
    Args:
        matrice_dist: Matrice de distances
        depart: Ville de départ
        
    Returns:
        Tuple (chemin, coût_total)
    """
    n = len(matrice_dist)
    visite = [False] * n
    chemin = [depart]
    visite[depart] = True
    cout_total = 0
    
    actuel = depart
    for _ in range(n - 1):
        # Trouver le plus proche voisin non visité
        distances = matrice_dist[actuel].copy()
        distances[visite] = np.inf  # Ignorer les villes visitées
        
        prochain = np.argmin(distances)
        chemin.append(prochain)
        visite[prochain] = True
        cout_total += matrice_dist[actuel][prochain]
        actuel = prochain
    
    # Retour au point de départ
    cout_total += matrice_dist[chemin[-1]][chemin[0]]
    
    return chemin, cout_total

# Test
chemin_nn, cout_nn = plus_proche_voisin(matrice_dist)
print(f"Chemin (Plus Proche Voisin): {chemin_nn}")
print(f"Coût total: {cout_nn:.2f}")

### Insertion la Plus Proche (Nearest Insertion)

In [ ]:
def insertion_plus_proche(matrice_dist: np.ndarray) -> Tuple[List[int], float]:
    """
    Heuristique d'insertion la plus proche.
    Commence avec un cycle de 2 villes, puis insère les autres une par une.
    """
    n = len(matrice_dist)
    
    # Initialiser avec les 2 villes les plus proches
    i, j = np.unravel_index(np.argmin(matrice_dist + np.eye(n) * np.inf), matrice_dist.shape)
    cycle = [i, j]
    non_visite = set(range(n)) - {i, j}
    
    while non_visite:
        meilleur_cout = float('inf')
        meilleur_ville = None
        meilleure_position = None
        
        # Pour chaque ville non visitée
        for ville in non_visite:
            # Trouver la meilleure position d'insertion dans le cycle
            for pos in range(len(cycle)):
                prev = cycle[pos - 1]
                curr = cycle[pos]
                cout_insertion = (matrice_dist[prev][ville] + 
                                matrice_dist[ville][curr] - 
                                matrice_dist[prev][curr])
                
                if cout_insertion < meilleur_cout:
                    meilleur_cout = cout_insertion
                    meilleure_ville = ville
                    meilleure_position = pos
        
        # Insérer la ville
        cycle.insert(meilleure_position, meilleure_ville)
        non_visite.remove(meilleure_ville)
    
    # Calculer le coût total
    cout_total = sum(matrice_dist[cycle[i]][cycle[i+1]] for i in range(len(cycle)-1))
    cout_total += matrice_dist[cycle[-1]][cycle[0]]
    
    return cycle, cout_total

# Test
chemin_ins, cout_ins = insertion_plus_proche(matrice_dist)
print(f"Chemin (Insertion Plus Proche): {chemin_ins}")
print(f"Coût total: {cout_ins:.2f}")

## 6. Métaheuristiques

### Recuit Simulé (Simulated Annealing)

**En Julia**, vous aviez probablement des packages comme `SimulatedAnnealing.jl`. En Python, nous l'implémentons nous-mêmes.

In [ ]:
def calculer_cout(chemin: List[int], matrice_dist: np.ndarray) -> float:
    """Calcule le coût total d'un chemin."""
    cout = sum(matrice_dist[chemin[i]][chemin[i+1]] for i in range(len(chemin)-1))
    cout += matrice_dist[chemin[-1]][chemin[0]]
    return cout

def voisin_2opt(chemin: List[int]) -> List[int]:
    """Génère un voisin en inversant une sous-séquence (2-opt)."""
    n = len(chemin)
    i, j = sorted(random.sample(range(1, n), 2))
    nouveau_chemin = chemin[:i] + chemin[i:j+1][::-1] + chemin[j+1:]
    return nouveau_chemin

def recuit_simule(matrice_dist: np.ndarray, 
                  solution_initiale: List[int],
                  temp_initiale: float = 1000.0,
                  facteur_refroidissement: float = 0.99,
                  iterations_par_temp: int = 100,
                  temp_minimale: float = 0.01) -> Tuple[List[int], float]:
    """
    Recuit simulé pour le TSP.
    
    Args:
        matrice_dist: Matrice de distances
        solution_initiale: Solution de départ
        temp_initiale: Température initiale
        facteur_refroidissement: Facteur de refroidissement (0 < facteur < 1)
        iterations_par_temp: Nombre d'itérations à chaque température
        temp_minimale: Température minimale (critère d'arrêt)
        
    Returns:
        Tuple (meilleur_chemin, meilleur_cout)
    """
    solution_courante = solution_initiale.copy()
    cout_courant = calculer_cout(solution_courante, matrice_dist)
    
    meilleure_solution = solution_courante.copy()
    meilleur_cout = cout_courant
    
    temperature = temp_initiale
    
    while temperature > temp_minimale:
        for _ in range(iterations_par_temp):
            # Générer un voisin
            voisin = voisin_2opt(solution_courante)
            cout_voisin = calculer_cout(voisin, matrice_dist)
            
            # Accepter ou rejeter
            delta = cout_voisin - cout_courant
            if delta < 0 or random.random() < np.exp(-delta / temperature):
                solution_courante = voisin
                cout_courant = cout_voisin
                
                # Mettre à jour la meilleure solution
                if cout_courant < meilleur_cout:
                    meilleure_solution = solution_courante.copy()
                    meilleur_cout = cout_courant
        
        # Refroidir
        temperature *= facteur_refroidissement
    
    return meilleure_solution, meilleur_cout

# Test avec solution initiale du plus proche voisin
print("Recuit simulé en cours...")
chemin_sa, cout_sa = recuit_simule(matrice_dist, chemin_nn, 
                                    temp_initiale=1000, 
                                    facteur_refroidissement=0.995,
                                    iterations_par_temp=50)
print(f"Chemin (Recuit Simulé): {chemin_sa}")
print(f"Coût total: {cout_sa:.2f}")
print(f"Amélioration par rapport à Plus Proche Voisin: {cout_nn - cout_sa:.2f}")

### Recherche Locale avec 2-opt

**En Julia**, vous aviez peut-être des fonctions de recherche locale. En Python, c'est similaire.

In [ ]:
def recherche_locale_2opt(chemin: List[int], matrice_dist: np.ndarray) -> Tuple[List[int], float]:
    """
    Recherche locale avec mouvements 2-opt jusqu'à convergence.
    """
    n = len(chemin)
    amelioration = True
    meilleur_chemin = chemin.copy()
    meilleur_cout = calculer_cout(meilleur_chemin, matrice_dist)
    
    while amelioration:
        amelioration = False
        
        for i in range(1, n - 1):
            for j in range(i + 1, n):
                if j - i == 1:
                    continue  # Ignorer les arêtes adjacentes
                
                # Nouveau chemin en inversant la section [i:j]
                nouveau_chemin = (meilleur_chemin[:i] + 
                                meilleur_chemin[i:j+1][::-1] + 
                                meilleur_chemin[j+1:])
                
                nouveau_cout = calculer_cout(nouveau_chemin, matrice_dist)
                
                if nouveau_cout < meilleur_cout:
                    meilleur_chemin = nouveau_chemin
                    meilleur_cout = nouveau_cout
                    amelioration = True
    
    return meilleur_chemin, meilleur_cout

# Test
print("Recherche locale 2-opt en cours...")
chemin_2opt, cout_2opt = recherche_locale_2opt(chemin_nn, matrice_dist)
print(f"Chemin (2-opt): {chemin_2opt}")
print(f"Coût total: {cout_2opt:.2f}")
print(f"Amélioration par rapport à Plus Proche Voisin: {cout_nn - cout_2opt:.2f}")

## 7. Visualisation des Solutions

### Visualisation d'un chemin TSP

In [ ]:
def visualiser_chemin(probleme, chemin: List[int], titre: str = "Solution TSP"):
    """
    Visualise un chemin TSP sur les coordonnées de l'instance.
    """
    # Obtenir les coordonnées si disponibles
    if hasattr(probleme, 'node_coords'):
        coords = {i+1: probleme.node_coords[i+1] for i in range(probleme.dimension)}
    else:
        print("⚠️  Pas de coordonnées disponibles pour cette instance")
        return
    
    # Extraire les coordonnées x et y
    x_coords = [coords[node][0] for node in chemin]
    y_coords = [coords[node][1] for node in chemin]
    
    # Fermer le cycle
    x_coords.append(x_coords[0])
    y_coords.append(y_coords[0])
    
    # Créer la figure
    plt.figure(figsize=(12, 8))
    plt.plot(x_coords, y_coords, 'b-', linewidth=2, alpha=0.6, label='Chemin')
    plt.scatter(x_coords[:-1], y_coords[:-1], c='red', s=100, zorder=5)
    
    # Annoter les villes
    for i, node in enumerate(chemin):
        plt.annotate(str(node), (x_coords[i], y_coords[i]), 
                    fontsize=8, ha='center', va='center')
    
    plt.title(titre, fontsize=14, fontweight='bold')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Tester la visualisation (si coordonnées disponibles)
try:
    visualiser_chemin(probleme, chemin_2opt, "Solution après 2-opt")
except Exception as e:
    print(f"Visualisation impossible: {e}")
    print("Utilisez afficherGraphe() pour visualiser le graphe complet")

## 8. Comparaison des Méthodes

### Tableau comparatif des résultats

In [ ]:
# Comparer toutes les méthodes testées
resultats = {
    "Plus Proche Voisin": cout_nn,
    "Insertion Plus Proche": cout_ins,
    "2-opt (recherche locale)": cout_2opt,
    "Recuit Simulé": cout_sa
}

# Afficher les résultats
print("=" * 50)
print("COMPARAISON DES MÉTHODES")
print("=" * 50)
for methode, cout in sorted(resultats.items(), key=lambda x: x[1]):
    print(f"{methode:30s}: {cout:10.2f}")

# Trouver la meilleure méthode
meilleure_methode = min(resultats.items(), key=lambda x: x[1])
print(f"\n🏆 Meilleure méthode: {meilleure_methode[0]} avec coût {meilleure_methode[1]:.2f}")

# Graphique comparatif
plt.figure(figsize=(10, 6))
methodes = list(resultats.keys())
couts = list(resultats.values())
couleurs = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']

bars = plt.bar(methodes, couts, color=couleurs, alpha=0.7, edgecolor='black')
plt.ylabel('Coût total', fontsize=12)
plt.title('Comparaison des méthodes de résolution TSP', fontsize=14, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)

# Ajouter les valeurs sur les barres
for bar, cout in zip(bars, couts):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{cout:.1f}',
             ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

## 9. Conseils pour la Transition Julia → Python

### Points clés à retenir

1. **Indexation** : Python utilise 0-based indexing (comme Julia), mais attention aux listes vs arrays
2. **Types** : Python est dynamiquement typé, mais utilisez des annotations de type pour la clarté
3. **Performance** : Utilisez NumPy pour les calculs numériques (comme les arrays Julia)
4. **Graphes** : NetworkX est très puissant, équivalent à `LightGraphs.jl` en Julia
5. **Optimisation** : PuLP pour PLNE (équivalent de JuMP), mais moins performant que JuMP

### Ressources utiles

- **Documentation NetworkX** : https://networkx.org/
- **Documentation PuLP** : https://pythonhosted.org/PuLP/
- **Documentation tsplib95** : https://tsplib95.readthedocs.io/
- **NumPy pour utilisateurs Julia** : https://numpy.org/doc/stable/user/numpy-for-matlab-users.html

### Prochaines étapes

1. Implémenter vos heuristiques existantes en Python
2. Adapter vos métaheuristiques (Recuit Simulé, Tabu Search, etc.)
3. Comparer les performances avec vos solutions Julia
4. Utiliser les visualisations pour analyser les solutions

## 10. Exercices Pratiques

### Exercice 1 : Implémenter une heuristique personnalisée
Créez votre propre heuristique constructive (ex: insertion la plus éloignée).

### Exercice 2 : Améliorer le recuit simulé
Testez différents paramètres (température initiale, facteur de refroidissement).

### Exercice 3 : Implémenter Tabu Search
Créez une recherche tabou pour améliorer encore les solutions.

### Exercice 4 : Comparer avec différentes instances
Testez vos méthodes sur plusieurs fichiers `.tsp` de tailles différentes.